# Lezione 15

Il materiale di riferimento per le lezioni a venire è costituito da:

* Il libro (di testo) [The Definitive ANTLR 4 Reference](https://pragprog.com/book/tpantlr2/the-definitive-antlr-4-reference) di Terence Parr (l'autore di ANTLR),
* le [informazioni su target/runtime Python 3](https://github.com/antlr/antlr4/blob/master/doc/python-target.md),
* i [sorgenti del runtime Python 3](https://github.com/antlr/antlr4/tree/master/runtime/Python3),
* le [API Java](https://www.antlr.org/api/Java/) (utili a colmare la scarsezza della documentazione dei target/runtime Python 3),
* la [versione Python 3 degli esempi di codice](https://github.com/jszheng/py3antlr4book) del libro di cui sopra (non ufficiale),
* un [insieme di grammatiche](https://github.com/antlr/grammars-v4) di diversi linguaggi noti.

## Uso "diretto" (senza LibLET)

In [ ]:
! rm -rf Hello*.*

In [ ]:
%%writefile Hello.g

grammar Hello;            // Define a grammar called Hello
r  : 'hello' ID ;         // match keyword hello followed by an identifier
ID : [a-z]+ ;             // match lower-case identifiers
WS : [ \t\r\n]+ -> skip ; // skip spaces, tabs, newlines, \r (Windows)

### Generazione dei moduli (usando il tool Java)

In [ ]:
! echo $ANTLR4_JAR

In [ ]:
! java -jar "$ANTLR4_JAR" -Dlanguage=Python3 -visitor Hello.g && ls -l Hello*py

### Parsing (costruzione dell'albero)

In [ ]:
# codice dal runtime di antlr4

from antlr4.CommonTokenStream import CommonTokenStream
from antlr4.InputStream import InputStream

In [ ]:
# codice genrerato da antlr4 a partire dalla grammatica Hello.g

from HelloLexer import HelloLexer
from HelloParser import HelloParser

In [ ]:
text = 'hello massimo'

lexer = HelloLexer(InputStream(text))
stream = CommonTokenStream(lexer)
parser = HelloParser(stream)
tree = parser.r()

tree.toStringTree(recog = parser)

#### Uso del lexer

In [ ]:
lexer.reset()

while True:
    token = lexer.nextToken()
    print(token)
    if token.type == token.EOF: break

### Un esempio maggiormente complesso

In [ ]:
! rm -f Expr*.*

In [ ]:
%%writefile Expr.g

grammar Expr; 

/** The start rule; begin parsing here. */

prog:   stat+ ;

stat:   expr NEWLINE
    |   ID '=' expr NEWLINE
    |   NEWLINE
    ;

expr:   expr ('*'|'/') expr
    |   expr ('+'|'-') expr
    |   INT
    |   ID
    |   '(' expr ')'
    ;

MUL :     '*' ;            // assigns token name to '*' used above in grammar
DIV :     '/' ;
ADD :     '+' ;
SUB :     '-' ;
ID  :     [a-zA-Z]+ ;      // match identifiers
INT :     [0-9]+ ;         // match integers
NEWLINE : '\r'? '\n' ;     // return newlines to parser (is end-statement signal)
WS :      [ \t]+ -> skip ; // toss out whitespace    

In [ ]:
! java -jar "$ANTLR4_JAR" -Dlanguage=Python3 -visitor Expr.g && ls -l Expr*py

In [ ]:
from ExprLexer import ExprLexer
from ExprParser import ExprParser

text = '2 * 3 + 4\n'

lexer = ExprLexer(InputStream(text))
stream = CommonTokenStream(lexer)
parser = ExprParser(stream)
tree = parser.prog()

tree.toStringTree(recog = parser)

#### Uso del Listener

In [ ]:
from antlr4.tree.Tree import ParseTreeWalker

In [ ]:
from ExprListener import ExprListener

class PrintListener(ExprListener):
    def exitEveryRule(self, ctx):
        print(ctx.start, ctx.stop, ExprParser.ruleNames[ctx.getRuleIndex()])

listener = PrintListener()
ParseTreeWalker.DEFAULT.walk(listener, tree)

#### Uso del Visitor

In [ ]:
from ExprVisitor import ExprVisitor

class PrintVisitor(ExprVisitor):
    def visitChildren(self, ctx):
        print(ctx.start, ctx.stop, ExprParser.ruleNames[ctx.getRuleIndex()])
        return super().visitChildren(ctx)
        
visitor = PrintVisitor()
visitor.visit(tree)

#### Intercettare gli errori

In [ ]:
from antlr4.error.ErrorListener import ErrorListener

class PrintLexerErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f'Lexer error: {line=}, {column=}, {msg=}, exception={e}')  
        
class PrintParserErrorListener(ErrorListener):
    def syntaxError(self, recognizer, offendingSymbol, line, column, msg, e):
        print(f'Parser error: {line=}, {column=}, {msg=}, exception={e}, rules={recognizer.getRuleInvocationStack()}')  

In [ ]:
text = '2 * 1 + !\n'

lexer = ExprLexer(InputStream(text))
lexer.removeErrorListeners()
lexer.addErrorListener(PrintLexerErrorListener())

stream = CommonTokenStream(lexer)

parser = ExprParser(stream)
parser.removeErrorListeners()
parser.addErrorListener(PrintParserErrorListener())

In [ ]:
tree = None
tree = parser.prog()

In [ ]:
tree.toStringTree(recog = parser)

## Uso mediato da LibLET

In [ ]:
from liblet import ANTLR

### Generazione e caricamento dei moduli

In [ ]:
LabeledExpr = ANTLR(r"""
grammar LabeledExpr; // rename to distinguish from Expr.g4

prog:   stat+ ;

stat:   expr NEWLINE                # printExpr
    |   ID '=' expr NEWLINE         # assign
    |   NEWLINE                     # blank
    ;

expr:   expr op=('*'|'/') expr      # MulDiv
    |   expr op=('+'|'-') expr      # AddSub
    |   INT                         # int
    |   ID                          # id
    |   '(' expr ')'                # parens
    ;

MUL :     '*' ;            // assigns token name to '*' used above in grammar
DIV :     '/' ;
ADD :     '+' ;
SUB :     '-' ;
ID  :     [a-zA-Z]+ ;      // match identifiers
INT :     [0-9]+ ;         // match integers
NEWLINE : '\r'? '\n' ;     // return newlines to parser (is end-statement signal)
WS :      [ \t]+ -> skip ; // toss out whitespace
""")

In [ ]:
# teniamola da parte per la prossima lezione…

LabeledExpr.save('LabeledExpr.g')

### Costruzione dei token e del parsing context

In [ ]:
text = '3 * 4 + 5\n'

In [ ]:
for token in LabeledExpr.tokens(text): 
  print(token)

In [ ]:
LabeledExpr.context(text, 'prog')

In [ ]:
# come stringa

LabeledExpr.context(text, 'prog', as_string = True)

### Generazione di un Tree ("annotato")

In [ ]:
# semplice

LabeledExpr.tree(text, 'prog', simple = True)

Si può ottenere un albero annotato, ossia un `Tree` i cui nodi siano `dict`. 

In tal caso ogni nodo contiene la chiave `type` che può valere `rule` o `token`, a seconda che il nodo corrisponda ad una regola della grammatica (context-free) o del tokenizer. 

Nel caso delle regole, sono presenti le chiavi `rule` (che ha per valore il nonterminale a sinistra nella regola) e `name` corrispondente dall'alternativa specifica della regola (se annotata nella grammatica). La chiave `src` contiene gli indici del primo ed ultimo token nel sorgente derivati dalla regola

Nel caso dei token, la chiave `name`  ha per valore il nome del token (se presente nella grammatica), mentre `value` ha per valore la porzione della parola in ingresso corrispondente al token. La chiave `src` contiene l'indice del token nel sorgente.

In [ ]:
LabeledExpr.tree(text, 'prog')

### Diagnostici

In [ ]:
Ambig = ANTLR(r"""
grammar Ambig;

stat: expr ';'
    | ID '(' ')' ';' 
    ;
expr: ID '(' ')' 
    | INT
    ;

INT : [0-9]+ ;
ID  : [a-zA-Z]+ ;
WS  : [ \t\r\n]+ -> skip ;
""")

text = 'f();'

In [ ]:
Ambig.context(text, 'stat', diag = True)

In [ ]:
Ambig.context(text, 'stat', trace = True)